In [ ]:
# %pip install simpletransformers

In [1]:
import torch
import pandas as pd
import numpy as np


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
merged_df = pd.read_parquet('processed_df/merged_df.parquet')

In [4]:
train_data = merged_df[['claim', 'wiki_text', 'wiki_title']]

In [5]:
train_data.rename(columns={'claim': 'query_text', 'wiki_text': 'gold_passage', 'wiki_title': 'title'}, inplace=True)

/tmp/ipykernel_3639148/144983162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={'claim': 'query_text', 'wiki_text': 'gold_passage', 'wiki_title': 'title'}, inplace=True)


In [6]:
train_data.head()

,query_text,gold_passage,title
0,nikolaj coster waldau worked with the fox broa...,nikolaj coster waldau lrb lsb ne ola k sd ald ...,Nikolaj_Coster-Waldau
1,nikolaj coster waldau was not in a danish thri...,nikolaj coster waldau lrb lsb ne ola k sd ald ...,Nikolaj_Coster-Waldau
2,nikolaj coster waldau worked with peter dinklage,nikolaj coster waldau lrb lsb ne ola k sd ald ...,Nikolaj_Coster-Waldau
3,nikolaj coster waldau refused to ever work wit...,nikolaj coster waldau lrb lsb ne ola k sd ald ...,Nikolaj_Coster-Waldau
4,nikolaj coster waldau was in a film,nikolaj coster waldau lrb lsb ne ola k sd ald ...,Nikolaj_Coster-Waldau


In [8]:
from simpletransformers.retrieval import RetrievalModel


model_type = "dpr"
context_encoder_name = "facebook/dpr-ctx_encoder-single-nq-base"
question_encoder_name = "facebook/dpr-question_encoder-single-nq-base"

model = RetrievalModel(
    model_type=model_type,
    context_encoder_name=context_encoder_name,
    query_encoder_name=question_encoder_name,
)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokeniz

In [9]:
hist=model.train_model(train_data, output_dir='/home/rahvk/data/tmp/cache/model', show_running_loss=True, use_cuda=True)
hist

Map:   0%|          | 0/102751 [00:00<?, ? examples/s]

Map:   0%|          | 0/102751 [00:00<?, ? examples/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/12844 [00:00<?, ?it/s]

/home/rahvk/data/tmp/anaconda3/envs/faiss_1.7.4/lib/python3.10/site-packages/simpletransformers/retrieval/retrieval_model.py:1659: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (max_idxs == torch.tensor(labels)).sum().cpu().detach().numpy().item()
/home/rahvk/data/tmp/anaconda3/envs/faiss_1.7.4/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


(12844, 0.2877897483785961)

In [ ]:
global_step, training_loss = hist

In [10]:
result = model.eval_model(train_data)
result

Map:   0%|          | 0/102751 [00:00<?, ? examples/s]

Map:   0%|          | 0/102751 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/102751 [00:00<?, ? examples/s]

  0%|          | 0/103 [00:00<?, ?it/s]

Map:   0%|          | 0/102751 [00:00<?, ? examples/s]

Map:   0%|          | 0/102751 [00:00<?, ? examples/s]

Running Evaluation:   0%|          | 0/12844 [00:00<?, ?it/s]

Retrieving docs:   0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
result

In [12]:
res, doc_ids, doc_vectors, doc_dicts = result

In [13]:
res

{'eval_loss': 8.43696731419875,
 'mrr@1': 0.33564636840517365,
 'mrr@2': 0.34746620470847,
 'mrr@3': 0.350992528864277,
 'mrr@5': 0.3539939595072878,
 'mrr@10': 0.3560110526093826,
 'top_1_accuracy': 0.33564636840517365,
 'top_2_accuracy': 0.3592860410117663,
 'top_3_accuracy': 0.36986501347918754,
 'top_5_accuracy': 0.3829549104144972,
 'top_10_accuracy': 0.3977868828527216}

In [19]:
doc_ids.shape

(102751, 10)

In [20]:
doc_ids[0]

array([ 3442., 14938.,  3440.,  3447., 69013.,  3448.,  3443., 14943.,
       69021., 14945.])